In [1]:
import os
# Enforces CPU-only execution of torch
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Configure environment to ensure single-threaded execution.
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"]= "1"
os.environ["OMP_NUM_THREADS"] = "1"

import torch
torch.set_num_threads(1)

In [2]:
from xtr.datasets import BEIR, BEIRDataset
from xtr.config import XTRConfig, XTRModel, XTRScaNNIndexConfig
from xtr.utils import canonical_index_name
from xtr.modeling.xtr import XTR

/future/u/scheerer/miniconda3/envs/xtr-eval/lib/python3.8/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
dataset = BEIRDataset(dataset=BEIR.SCIFACT, datasplit="test")
index_config = XTRScaNNIndexConfig()
index_name = canonical_index_name(dataset=dataset, index_config=index_config)
config = XTRConfig(index_name=index_name, model=XTRModel.BASE_EN, index_config=index_config, override=False)

In [4]:
xtr = XTR(config=config, collection=dataset.collection, device=torch.device("cpu"))

  0%|          | 0/5183 [00:00<?, ?it/s]

#> Preparing corpus for BEIR BEIR.SCIFACT/test


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:00<00:00, 354495.43it/s]


Loading existing index from /future/u/scheerer/home/data/xtr-eval/indexes/BEIR.SCIFACT.split=test.XTRIndexType.SCANN.


[libprotobuf WARNING external/com_google_protobuf/src/google/protobuf/text_format.cc:339] Warning parsing text-format research_scann.ScannConfig: 38:5: text format contains deprecated field "min_cluster_size"


In [9]:
rankings = xtr.retrieve_docs(dataset.queries, document_top_k=100, token_top_k=1000)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:04<00:00,  4.63it/s]


In [10]:
dataset.eval(rankings)

[Result]
ndcg_cut@5: 0.6898
ndcg_cut@10: 0.7145
ndcg_cut@50: 0.7305
ndcg_cut@100: 0.7347
map_cut@5: 0.6583
map_cut@10: 0.6707
map_cut@50: 0.6745
map_cut@100: 0.6749
recall@5: 0.7648
recall@10: 0.8363
recall@50: 0.9043
recall@100: 0.9310
